## Transfer Learning usando dados MNIST
Para ilustrar o poder e o conceito de transfer learning, treinaremos uma CNN apenas nos dígitos 5,6,7,8,9. Em seguida, treinaremos apenas a(s) última(s) camada(s) da rede nos dígitos 0,1,2,3,4 e veremos quão bem as características aprendidas nos dígitos 5-9 ajudam na classificação dos dígitos 0-4.

Adaptado do Exemplo do Keras.

In [ ]:
# Preliminares

from __future__ import absolute_import, division, print_function  # Compatibilidade Python 2/3

import warnings
warnings.filterwarnings("ignore")

import datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
## Importar objetos do Keras para Deep Learning

import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

In [ ]:
# usado para ajudar algumas das funções de tempo
now = datetime.datetime.now

In [ ]:
# definir alguns parâmetros
batch_size = 128
num_classes = 5
epochs = 5

In [ ]:
# definir mais alguns parâmetros
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

In [ ]:
## Isso apenas lida com alguma variabilidade na forma como os dados de entrada são carregados

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [ ]:
## Para simplificar as coisas, escreva uma função para incluir todas as etapas de treinamento
## Como entrada, a função recebe um modelo, conjunto de treinamento, conjunto de teste e o número de classes
## Dentro do objeto modelo estará o estado sobre quais camadas estamos congelando e quais estamos treinando

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('formato de x_train:', x_train.shape)
    print(x_train.shape[0], 'amostras de treino')
    print(x_test.shape[0], 'amostras de teste')

    # converter vetores de classe em matrizes de classe binárias
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Tempo de treinamento: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Pontuação no teste:', score[0])
    print('Precisão no teste:', score[1])

In [ ]:
# os dados, embaralhados e divididos entre conjuntos de treino e teste
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# criar dois conjuntos de dados: um com dígitos abaixo de 5 e outro com 5 e acima
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [ ]:
# Definir as camadas de "características". Essas são as camadas iniciais que esperamos que "transfiram"
# para um novo problema. Congelaremos essas camadas durante o processo de fine-tuning

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [ ]:
# Definir as camadas de "classificação". Essas são as camadas posteriores que predizem as classes específicas a partir das características
# aprendidas pelas camadas de características. Esta é a parte do modelo que precisa ser re-treinada para um novo problema

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [ ]:
# Criamos nosso modelo combinando os dois conjuntos de camadas da seguinte forma
model = Sequential(feature_layers + classification_layers)

In [ ]:
# Vamos dar uma olhada
model.summary()

In [ ]:
# Agora, vamos treinar nosso modelo nos dígitos 5,6,7,8,9

train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

### Congelando Camadas
O Keras permite que as camadas sejam "congeladas" durante o processo de treinamento. Ou seja, algumas camadas teriam seus pesos atualizados durante o processo de treinamento, enquanto outras não. Esta é uma parte central do transfer learning, a capacidade de treinar apenas a última ou várias camadas.

Observe também que muito do tempo de treinamento é gasto "retropropagando" os gradientes de volta para a primeira camada. Portanto, se precisarmos apenas calcular os gradientes de volta a um pequeno número de camadas, o tempo de treinamento é muito mais rápido por iteração. Isso além das economias obtidas por poder treinar em um conjunto de dados menor.

In [ ]:
# Congelar apenas as camadas de características
for l in feature_layers:
    l.trainable = False

Observe abaixo as diferenças entre o número de *parâmetros totais*, *parâmetros treináveis* e *parâmetros não-treináveis*.

In [ ]:
model.summary()

In [ ]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

Observe que após uma única época, já estamos alcançando resultados na classificação de 0-4 que são comparáveis àqueles alcançados em 5-9 após 5 épocas completas. Isso apesar do fato de que estamos apenas "ajustando" a última camada da rede, e todas as camadas iniciais nunca viram como os dígitos 0-4 se parecem.

Além disso, observe que mesmo que quase todos (590K/600K) dos *parâmetros* fossem treináveis, o tempo de treinamento por época ainda foi muito reduzido. Isso ocorre porque a parte não congelada da rede era muito superficial, tornando a retropropagação mais rápida.

## Exercício
### Para fazer:
- Agora escreva código para reverter este processo de treinamento. Ou seja, você treinará nos dígitos 0-4, e então fará fine-tune apenas das últimas camadas nos dígitos 5-9.